# Chapter 15 – Weather Prediction
### <font color=blue>Traditional Linear Model that predicts temperature(s) using previous temperatures data</font>

## Import Modules

In [1]:
# Common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#tensorflow imports
import tensorflow as tf
from tensorflow import keras

#sklearn imports
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Function Definitions

In [2]:
#function to verify the existence of a file in the current working directory and download it if not
import os,urllib, urllib.request, sys, tarfile
def downloadDataResource(file, sourcePath, compressed=None):
    if not os.path.isfile(file):
        try:
            urllib.request.urlretrieve(sourcePath+(compressed if compressed else file),(compressed if compressed else file))
            print("Downloaded", (compressed if compressed else file) )
            if compressed:
                ucomp = tarfile.open(compressed)
                ucomp.extractall()
                ucomp.close()
                print("File uncompressed.")
        except:
            print("ERROR: File", (compressed if compressed else file), "not found. Data source missing.")
    else:
        print("Data resource", file, "already downloaded.")

In [3]:
#function provided that plots the learning curve for neural networks
def nn_plot_learning_curve( history ):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    ymin, ymax = [], []
    for x in history.history.keys():
        ymax.append( max(history.history[x]))
        ymin.append( min(history.history[x]))
    plt.gca().set_ylim(min(ymin)*.95, max(ymax)*1.05)
    plt.xlabel("EPOCHS")
    plt.show()

In [4]:
#function to plot actual values vs. predicted values
def plot_actual_pred( actual, prediction ):
    plt.plot(actual, ".-", alpha=.6, label="Actual")
    plt.plot(prediction, ".-", alpha=.6, label="Prediction")
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
#function that shows a learning curve for any model that has predict or fit methods
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator,X,y,ylim=None,cv=None,n_jobs=None,train_sizes=np.linspace(0.1, 1.0, 20),scoring = 'neg_root_mean_squared_error'):
    
    _, axes = plt.subplots(1, 1, figsize=(10, 5))    
    axes.set_title('Learning Curve')
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel(scoring)

    train_sizes, train_scores, test_scores= learning_curve(estimator,X,y,cv=cv,n_jobs=n_jobs,train_sizes=train_sizes,scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes,train_scores_mean - train_scores_std,train_scores_mean + train_scores_std,alpha=0.1,color="r")
    axes.fill_between(train_sizes,test_scores_mean - test_scores_std,test_scores_mean + test_scores_std,alpha=0.1,color="g")
    axes.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    axes.plot(train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score")
    axes.legend(loc="best")
    plt.show()
    
    return

#code to prevent warnings that can occur as a result of this function
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

## Source Data

In [6]:
#download data files if not currently found in your datasets directory (under the current working directory)
path = 'https://raw.githubusercontent.com/SueMcMetzger/MachineLearning/main/chpt15/'
filename = "VillanovaUniversityWeather.csv"

downloadDataResource(filename,path)

#create a dataframe with the data from the CSV file
data = pd.read_csv(filename)

Data resource VillanovaUniversityWeather.csv already downloaded.


In [7]:
#convert the date information to a date value
data['date'] = data['dt_iso'].apply(lambda x: x[0:20])
data['date']= pd.to_datetime(data['date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [8]:
#remove unnecessary data (but feel free to add back in data if you wish
data.drop(['dt_iso', 'rain_1h','rain_3h','snow_1h','snow_3h','temp_min','temp_max','clouds_all','weather_id','weather_main','weather_description'],axis=1,inplace=True)

## Filter Data (for performance)

In [9]:
#filter data so we are only seeing the temperature at 1 given hour in the day
#choose hour 5 which in EST time zone would be at mid-day
#feel free to comment out this filter and you will have hourly data to anlayze (as opposed to daily)
data = data[data['date'].dt.hour == 5]

#OR limited data to 2 years worth - feel free to change this if you want to look at more years
data = data[data['date'].dt.year > 2018 ]

In [10]:
#add columns
#data['hour'] = data.date.dt.hour  # NOT INCLUDING when we filter on hour (because column will have the same value)
data['year'] = data.date.dt.year
data['month'] = data.date.dt.month
data['day'] = data.date.dt.day

## Prepare the Data

In [11]:
#set the date field as the index
data.set_index('date',inplace = True)
data.drop_duplicates(inplace=True)  #There appears to be some duplicates!

In [12]:
#let's look at the first few instances
data.head()

,temp,feels_like,pressure,humidity,wind_speed,wind_deg,year,month,day
date,,,,,,,,,
2019-01-01 05:00:00,46.51,43.81,1008,100,3.36,200,2019,1,1
2019-01-02 05:00:00,40.37,34.30,1023,75,4.70,295,2019,1,2
2019-01-03 05:00:00,39.16,32.68,1007,86,6.08,160,2019,1,3
2019-01-04 05:00:00,30.07,23.77,1011,100,4.36,247,2019,1,4
2019-01-05 05:00:00,42.22,38.01,996,100,4.32,43,2019,1,5


### Save FUTURE data points to predict

In [13]:
FUTURE = 5

#For demonstrations purposes, keep the predictions aside so you can evaluate the results
toPredict = data[-FUTURE:].copy()
y_predict = list(toPredict.temp)

data = data[:-FUTURE].copy()

#### Use the columns to create both the test and the training data sets

In [14]:
#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['temp']), 
    data.temp,
    test_size=0.2,
    random_state=32,
    stratify = data.year
)
X_train.shape,  y_train.shape, X_test.shape,  y_test.shape

((581, 8), (581,), (146, 8), (146,))

In [15]:
#no categorical attributes for this data set (nice to have in case data changes)
cat_attribs = []

#set the numerical attributes
num_attribs = list( data.drop(columns=['temp']) )

#define pipeline for numeric attributes (this code is just a definition)
#each numeric attribute will be imputated using the Median strategy
#each numeric attribute will be scaled 
num_pipeline = Pipeline( [
    ('imputer', SimpleImputer(strategy="median")), #because no missing values, not used
    ('std_scaler', StandardScaler()),   
] )

#define the pipeline process for the data set
full_pipeline = ColumnTransformer( [
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(sparse=False), cat_attribs)      #because no categorical attributes, not used 
])

In [16]:
#create an array of prepared data based on the training data set
X_train = full_pipeline.fit_transform( X_train)
X_train.shape

(581, 8)

In [17]:
#create an array of prepared data based on the test data set
X_test = full_pipeline.transform( X_test)
X_test.shape

(146, 8)

In [18]:
X_predict = full_pipeline.transform( toPredict )
X_predict.shape

ValueError: X has 9 features, but ColumnTransformer is expecting 8 features as input.

## Computing Some Baselines

<font color=blue>Naive Forecasting</font>: just use the last observed value in the series to predict the next value in the series

In [ ]:
#calculate the average home value
baseline_prediction = y_train.mean()

#populate an array with the average home value
predictions = np.full(shape=len(X_train), fill_value = baseline_prediction)

#determine the Root Mean Squared Error based on the actual vs. the baseline prediction
baseline_rmse = mean_squared_error(y_train, predictions, squared=False)
print("Baseline temp: {:,.2f}".format(baseline_prediction))
print("Baseline Performance (of this guess): {:,.2f}".format(baseline_rmse))

## Linear Regression Model

In [ ]:
#create the model object
model = Ridge(alpha=0.5, solver='saga')

#fit the model to the prepared test data
model.fit(X_train,y_train)

#calculate the predicted values
y_pred = model.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)
print("Prediction Error (RMSE): {:,.4f}".format(rmse))

In [ ]:
#use cross valudation to process the data 10 different ways using linear regression model generated above
#helps us to understand how well the model "fits" our data
scores = cross_val_score(model, X_train, y_train,
                         scoring="neg_root_mean_squared_error", cv=10)

#calculate the average score over the 10 different cross validations
print("Average of RMSE across folds: {:,.4f}".format(-scores.mean()))

In [ ]:
plot_learning_curve(model, X_train, y_train)

In [ ]:
#calculate the predicted values for Test
y_pred = model.predict(X_test)

lin_rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Test Prediction Error (RMSE): {:,.4f}".format(lin_rmse))

In [ ]:
#calculate the predicted values for Test
y_pred = model.predict(X_predict)

lin_rmse = mean_squared_error(y_predict, y_pred, squared=False)
print("Test Prediction Error (RMSE): {:,.4f}".format(lin_rmse))

In [ ]:
plot_actual_pred(y_predict, y_pred)

## SGD Regressor

In [ ]:
#create a Stocahstic Gradiant Descent Regressor object
model = SGDRegressor(max_iter=1000, tol=.01, penalty="l2", eta0=.001)

#fit the model to the training data
model.fit(X_train, y_train)

#calcualte the predicted values
predictions = model.predict(X_train)

#compare the predicted to the actuals to evaluate the model
rmse = mean_squared_error(y_train,predictions,squared=False)
print("Predition Error (RMSE): {:,.4f}".format(rmse))

In [ ]:
plot_learning_curve(model, X_train, y_train)

In [ ]:
#calculate the predicted values for Test
y_pred =model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Test Prediction Error (RMSE): {:,.4f}".format(rmse))

In [ ]:
#calculate the predicted values for Test
y_pred = model.predict(X_predict)

rmse = mean_squared_error(y_predict, y_pred, squared=False)
print("Test Prediction Error (RMSE): {:,.4f}".format(rmse))

In [ ]:
plot_actual_pred(y_predict, y_pred)